In [ ]:
from astropy.io import fits
from astropy.table import unique, vstack
from astropy.table import Table
from astropy.time import Time
from astroquery.mast import Mast,Observations
import matplotlib.pyplot as plt
import os
import pandas as pd
import numpy as np
from glob import glob
import json

In [ ]:
proposal_obs = Observations.query_criteria(obstype='all',obs_collection='JWST',instrument_name='NIRCAM/CORON')
#print("Number of observations:",len(proposal_obs))
print(proposal_obs)

In [ ]:
batch_size = 10
batches = [proposal_obs[i:i+batch_size] for i in range(0, len(proposal_obs), batch_size)]
t = [Observations.get_product_list(obs) for obs in batches]
products = unique(vstack(t), keys='productFilename')
filtered_products = Observations.filter_products(products)

In [ ]:
prd_df = filtered_products.to_pandas()

In [ ]:
prd_df['proposal_id'].unique()

In [ ]:
df = prd_df.loc[prd_df['dataRights'] != 'EXCLUSIVE_ACCESS']

In [ ]:
df['proposal_id'].unique()

In [ ]:
#jw = level3_df[(level3_df['proposal_id']=='2278') & (level3_df['calib_level'] == 2) & (level3_df['productType'] == 'SCIENCE') & ((level3_df['productSubGroupDescription'] == 'CALINTS'))]

In [ ]:
#jw = level3_df[(level3_df['proposal_id']=='1075')  & (level3_df['productType'] == 'SCIENCE') & (level3_df['calib_level'] == 2) & (level3_df['productSubGroupDescription'] == 'RATEINTS')]

In [ ]:
#len(jw)

In [ ]:
#products = Table.from_pandas(jw)

In [ ]:
def save_products(df,programs):

    for program in programs:
        
        jw = df[(df['proposal_id']==program)  & (df['productType'] == 'SCIENCE') & (df['calib_level'] == 2) & (df['productSubGroupDescription'] == 'RATEINTS')]
        products = Table.from_pandas(jw)
        Observations.download_products(products,mrp_only=False,download_dir=f'/home/sarperyn/sarperyurtseven/ProjectFiles/dataset/NIRCAM/{program}')
        print(f'{program} is done.')
        

In [ ]:
programs = ['1075','1184','1194','1386','1411']

In [ ]:
save_products(df=df,programs=programs)

In [ ]:
def determine_and_count(column:str,table):    
    prop_list = []
    counter = {}
    for i in range(len(table['proposal_id'])):

        if table[f'{column}'][i] not in prop_list:

            prop_list.append(table[f'{column}'][i])
            try:
                counter[table[f'{column}'][i]] = 1
            except:
                #print(table[f'{column}'][i])
                continue
        else:
            try:
                counter[table[f'{column}'][i]] += 1
            except:
                continue
            
    print(counter)

In [ ]:
determine_and_count('calib_level',products)

In [ ]:
Observations.download_products(products,mrp_only=False,download_dir='/home/sarperyn/sarperyurtseven/ProjectFiles/dataset/NIRCAM/1075')

In [ ]:
# Path containing the rateints data
directory = f'/home/sarperyn/HST/mastDownload/HST/'

In [ ]:
fits_files = glob(os.path.join(directory, '**/*.fits'))

In [ ]:
fits_files

In [ ]:
f1 = fits.open(fits_files[10])

In [ ]:
def get_sci(fits_):
    
    header = []
    sci_data = []
    
    for i,path in enumerate(fits_):
        fits_file = fits.open(path)
        hd = fits_file[0].header
        sci = fits_file[1].data
        
        header.append(hd)
        sci_data.append(sci)
        
    return header,sci_data

In [ ]:
headers, scis = get_sci(fits_files)

In [ ]:
headers[0]

In [ ]:
plt.figure(figsize=(15,5))
plt.imshow(scis[0],clim=(0,250),cmap='gray')

In [ ]:
plt.figure(figsize=(15,5))
plt.imshow(scis[14],clim=(0,250),cmap='gray')